### 1.Importation les librairies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb

## 2. Importation un fichier

In [2]:
data=pd.read_csv("Farmer.csv",sep=";")

## 3.Verifier les donnees manquantes

In [3]:
data.isnull().sum()

 ID                               0
Parent ID                         2
Object state                      0
Status                            0
Agent name                        0
                               ... 
Latest soybean selling price      0
Time to payment                   6
Interest for rapid payment       63
Harvest pick-up conditions      187
Need for rapid pick-up          188
Length: 79, dtype: int64

### Affichage des colonnes avec plus de données manquantes

In [4]:
columns = list(data.columns)
many_missing_data_df = dict()
columns_with_many_missing_data = []

for column in columns:
    missing = data[column].isnull().sum() 
    missing_pcnt = round(missing * 100 / data.shape[0], 2)
    if missing_pcnt > 80:
        many_missing_data_df[column] = str(missing_pcnt)  + "%"
        columns_with_many_missing_data.append(column)

many_missing_data_df = pd.DataFrame({"Pourcentage de données manquantes": many_missing_data_df})
many_missing_data_df

,Pourcentage de données manquantes
Education level,81.84%
"If other sorghum type, precise",95.24%
"If other soybean type, precise",99.42%
"If yes, capacity to pay",80.68%
Subtitle,100.0%


## 4. Verifier les types de variables

In [5]:
data.dtypes

 ID                               int64
Parent ID                       float64
Object state                     object
Status                           object
Agent name                       object
                                 ...   
Latest soybean selling price      int64
Time to payment                  object
Interest for rapid payment       object
Harvest pick-up conditions       object
Need for rapid pick-up           object
Length: 79, dtype: object

## convertir ces types de variables

In [6]:
data = data.convert_dtypes()

## Voir si ce sont converties

In [7]:
data.dtypes

 ID                              Int64
Parent ID                        Int64
Object state                    string
Status                          string
Agent name                      string
                                 ...  
Latest soybean selling price     Int64
Time to payment                 string
Interest for rapid payment      string
Harvest pick-up conditions      string
Need for rapid pick-up          string
Length: 79, dtype: object

## Details sur notre dataFrame

In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 79 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0    ID                             1030 non-null   Int64  
 1   Parent ID                       1028 non-null   Int64  
 2   Object state                    1030 non-null   string 
 3   Status                          1030 non-null   string 
 4   Agent name                      1030 non-null   string 
 5   Agent first name                1030 non-null   string 
 6   Title                           1030 non-null   string 
 7   Subtitle                        0 non-null      Int64  
 8   Lon                             1029 non-null   Float64
 9   Lat                             1029 non-null   Float64
 10  Accuracy                        1030 non-null   Int64  
 11  Date                            1030 non-null   string 
 12  Gender                          10

## 5.Nettoyage des donnees

###  normalisation des noms des colonnes

In [18]:
datacopy = data.copy()

In [19]:
from re import sub

def format_column_name(column_name):
    column_name = column_name.strip().lower()
    column_name = column_name.replace(' ', '_')
    column_name = column_name.replace('-', '_')

    return sub(r'[()?<>./,]', '', column_name).replace('-', '_')

columns = list(datacopy.columns)
for column in columns:
    datacopy.rename(columns= {column: format_column_name(column)}, inplace=True)

###  distinguer les variables quantitatives et qualificatives

In [20]:
cat_data = []
num_data = []
for i, c in enumerate (datacopy.dtypes):
    cat_data.append(datacopy.columns[i]) if c == "string"  else num_data.append(datacopy.columns[i])      

print("Les Variables quantitatives sont: \n", num_data)
print("\nLes Variables qualificatives sont: \n", cat_data)

Les Variables quantitatives sont: 
 ['id', 'parent_id', 'subtitle', 'lon', 'lat', 'accuracy', 'age', 'phone_number', 'number_of_years_in_activity', 'year_of_membership', 'number_of_spouses', 'number_of_children', 'nb_children__6_yrs_old', 'nb_children_between_6_18_yrs', 'nb_children_attending_school', 'annual_income', 'total_number_of_farming_plots', 'total_farm_area', 'total_sorghum_area', 'total_soybean_area', 'average_loan_amount', 'loan_rate', 'ongoing_bridging_credit', 'amount_repayable_in_cash', 'if_yes_for_which_amount', 'actual_spending_on_inputs', 'latest_sorghum_selling_price', 'latest_soybean_selling_price']

Les Variables qualificatives sont: 
 ['object_state', 'status', 'agent_name', 'agent_first_name', 'title', 'date', 'gender', 'last_family_name', 'first_given_name', 'literate', 'education_level', 'farmer_picture', 'id_card_picture', 'spouses_involved_in_trade', 'spouses_involved_in_food_crops', 'spouses_involved_in_craftwork', 'nb_children_working_on_farm', 'means_of_tr

### remplacer les données vides des variables qualificatives par la mode

In [21]:
for column in cat_data:
    datacopy[column].fillna(datacopy[column].mode()[0], inplace=True)

### verification des modalites

In [22]:
for i in datacopy.columns:
    print("colonne:",i)
    print(datacopy[i].unique())
    print("\n")

colonne: id
<IntegerArray>
[  73,   74,   75,   76,   77,   78,   79,   88,   89,   90,
 ...
 1047, 1048, 1049, 1050, 1051, 1052, 1053, 1054, 1055, 1056]
Length: 1030, dtype: Int64


colonne: parent_id
<IntegerArray>
[  73,   86,   95,   99,   77,   84,   96,   98,  105,   75,   81,   87,   88,
   90,   92,   97,  103,  104,  107,  109,  113,   79,  108,  114,   76,   89,
  101,  102,  115,   78, <NA>,   85,   74,   82,   93,   94,  111,   80,   91,
  100,  106,  112]
Length: 42, dtype: Int64


colonne: object_state
<StringArray>
['Validated']
Length: 1, dtype: string


colonne: status
<StringArray>
['Enable']
Length: 1, dtype: string


colonne: agent_name
<StringArray>
[   'ATIBUGRI',     'Braimah', 'Abdul-Razak',       'Hamza',    'Mohammed',
      'Ziblim',      'Yahaya',       'Azure',     'Abukari']
Length: 9, dtype: string


colonne: agent_first_name
<StringArray>
['ABUGBIL GEORGE',       'Bushira ',          'Amin ',   'Paul Fuseini',
         'Wumbei',           'Nuhu',        

### Suppression des colonnes qui ont une modalité unique

In [23]:
datacopy.drop(columns=["status"], inplace=True)

datacopy.drop(columns=["object_state"], inplace=True)

### Supprimer des les lignes avec les doublons

In [24]:
datacopy.drop_duplicates()

,id,parent_id,agent_name,agent_first_name,title,subtitle,lon,lat,accuracy,date,...,interest_personalized_advisory,if_yes_capacity_to_pay1,main_buyer,search_for_buyer,latest_sorghum_selling_price,latest_soybean_selling_price,time_to_payment,interest_for_rapid_payment,harvest_pick_up_conditions,need_for_rapid_pick_up
0,73,73,ATIBUGRI,ABUGBIL GEORGE,Issifu - Abugbilla,<NA>,-0.133757,10.799995,2,2017-11-28 16:54:12,...,Yes,Between 5 and 15 Kg/acr,Market,Easy,5,0,Within 24 hours,Upon delivery,Good,Useful
1,74,73,ATIBUGRI,ABUGBIL GEORGE,Akudug - Anyanr,<NA>,-0.132925,10.798753,2,2017-11-21 10:21:58,...,No,5 Kg/acr,Market,Easy,5,0,Within 24 hours,Upon delivery,Good,Useful
2,75,73,ATIBUGRI,ABUGBIL GEORGE,Ateni - Awindaug,<NA>,-0.133783,10.799805,2,2017-12-05 10:46:22,...,Yes,5 Kg/acr,Market,Easy,3,0,Within 24 hours,Upon delivery,Good,Useful
3,76,73,ATIBUGRI,ABUGBIL GEORGE,Ndewin - Amua,<NA>,-0.135947,10.801207,2,2017-11-21 10:01:33,...,May be,Between 5 and 15 Kg/acr,Market,Difficult,2,0,Within 24 hours,Upon delivery,Good,Useful
4,77,73,ATIBUGRI,ABUGBIL GEORGE,Amadim - Agingiri,<NA>,-0.133848,10.801405,2,2017-11-27 09:23:36,...,Yes,Between 15 and 30 kg/acr,Cooperative,Easy,2,0,Within 24 hours,Upon delivery,Good,Useful
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025,1052,112,Abukari,Silas,Issah - Alhassan,<NA>,-0.865843,9.604827,2,2017-12-02 18:48:10,...,Yes,Between 5 and 15 Kg/acr,Market,Very difficult,0,5,Within 24 hours,Within 24 hours after delivery,Average,Moderately useful
1026,1053,112,Abukari,Silas,Sulemana - Abdulai,<NA>,-0.865812,9.604847,2,2017-12-02 18:46:07,...,Yes,5 Kg/acr,Market,Very difficult,0,5,Within 24 hours,Within 48 hours after delivery,Average,Moderately useful
1027,1054,112,Abukari,Silas,Salifu - Tahiru,<NA>,-0.864513,9.60147,2,2017-12-02 19:11:38,...,No,5 Kg/acr,Market,Very difficult,0,5,Within 48 hours,Within 24 hours after delivery,Average,Moderately useful
1028,1055,112,Abukari,Silas,Seidu - Mahama,<NA>,-0.864513,9.60147,2,2017-12-02 19:12:07,...,Yes,5 Kg/acr,Market,Difficult,0,5,Within 48 hours,Within 24 hours after delivery,Average,Useful


In [26]:
pd.crosstab(datacopy["agent_name"],datacopy["lat"],margins="True")

lat,0.0,9.30062333333333,9.302455,9.31004,9.31635666666667,9.31635833333333,9.31641333333333,9.31650333333333,9.31748,9.32328666666667,...,10.87178,10.8718383333333,10.871885,10.87215,10.8725916666667,10.874831666667,10.876008333333,10.876136666667,10.876278333333,All
agent_name,,,,,,,,,,,,,,,,,,,,,
ATIBUGRI,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,115
Abdul-Razak,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,100
Abukari,0,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,157
Azure,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,135
Braimah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,71
Hamza,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,107
Mohammed,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,96
Yahaya,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,99
Ziblim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,149
